In [1]:
import numpy as np
import pandas as pd

In [2]:
from typing import List, Dict

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
songs = pd.read_csv('songdata.csv')

In [5]:
songs.head()


,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [6]:
songs = songs.sample(n=5000).drop('link', axis=1).reset_index(drop=True)


In [7]:
songs['text'] = songs['text'].str.replace(r'\n', '')

C:\Users\HP\AppData\Local\Temp\ipykernel_26972\269676193.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  songs['text'] = songs['text'].str.replace(r'\n', '')


In [8]:
tfidf = TfidfVectorizer(analyzer='word', stop_words='english')

In [9]:
lyrics_matrix = tfidf.fit_transform(songs['text'])

In [10]:
cosine_similarities = cosine_similarity(lyrics_matrix) 

In [11]:
similarities = {}

In [12]:
for i in range(len(cosine_similarities)):
    # Now we'll sort each element in cosine_similarities and get the indexes of the songs. 
    similar_indices = cosine_similarities[i].argsort()[:-50:-1] 
    # After that, we'll store in similarities each name of the 50 most similar songs.
    # Except the first one that is the same song.
    similarities[songs['song'].iloc[i]] = [(cosine_similarities[i][x], songs['song'][x], songs['artist'][x]) for x in similar_indices][1:]

In [13]:
class ContentBasedRecommender:
    def __init__(self, matrix):
        self.matrix_similar = matrix

    def _print_message(self, song, recom_song):
        rec_items = len(recom_song)
        
        print(f'The {rec_items} recommended songs for {song} are:')
        for i in range(rec_items):
            print(f"Number {i+1}:")
            print(f"{recom_song[i][1]} by {recom_song[i][2]} with {round(recom_song[i][0], 3)} similarity score") 
            print("--------------------")
        
    def recommend(self, recommendation):
        # Get song to find recommendations for
        song = recommendation['song']
        # Get number of songs to recommend
        number_songs = recommendation['number_songs']
        # Get the number of songs most similars from matrix similarities
        recom_song = self.matrix_similar[song][:number_songs]
        # print each item
        self._print_message(song=song, recom_song=recom_song)

In [14]:
recommedations = ContentBasedRecommender(similarities)

In [15]:
recommendation = {
    "song": songs['song'].iloc[10],
    "number_songs": 4 
}

In [16]:
recommedations.recommend(recommendation)

The 4 recommended songs for TV Eye are:
Number 1:
01.Tv Makes The Superstar by Modern Talking with 0.458 similarity score
--------------------
Number 2:
Cat by Bob Seger with 0.449 similarity score
--------------------
Number 3:
Cable Tv by Weird Al Yankovic with 0.386 similarity score
--------------------
Number 4:
Tv Casualty by Misfits with 0.304 similarity score
--------------------


In [17]:
recommendation2 = {
    "song": songs['song'].iloc[120],
    "number_songs": 4 
}

In [18]:
recommedations.recommend(recommendation2)

The 4 recommended songs for In The Lap Of The Gods are:
Number 1:
Alone Again In The Lap Of Luxury by Marillion with 0.29 similarity score
--------------------
Number 2:
Save Us All by Tracy Chapman with 0.192 similarity score
--------------------
Number 3:
I Wanna Be Where You Are by Michael Jackson with 0.147 similarity score
--------------------
Number 4:
Stranded by Heart with 0.146 similarity score
--------------------
